In [3]:
from typing import Literal, Annotated, Sequence, TypedDict
from typing_extensions import Annotated, TypedDict
from langchain_core.pydantic_v1 import BaseModel, Field
from langgraph.graph import START, END, StateGraph
from langgraph.prebuilt import ToolNode
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph.message import add_messages
from langchain.schema import SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate
from dotenv import load_dotenv
from langchain.tools.retriever import create_retriever_tool
from langchain_pinecone import PineconeVectorStore
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

load_dotenv()
llm = ChatOpenAI(model="gpt-4o-mini")
database = PineconeVectorStore.from_existing_index(
  index_name="vtw-pdf-openai", 
  embedding=OpenAIEmbeddings(model="text-embedding-3-large"),
)
retriever = database.as_retriever()

class RouteQuery(BaseModel):
    datasource: Literal["vectorstore", "own-generating"] = Field(
        ...,
        description="Given a user question choose to route it to own-generating or a vectorstore.",
    )

structured_llm_router = llm.with_structured_output(RouteQuery)

system = """You are an expert at routing a user question to a vectorstore or not
The vectorstore contains documents related to IT company VTW.
If there are questions about 'VTW', '브띠따', '브이티더블유', Use the vectorstore for questions."""
route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

test = route_prompt | structured_llm_router

print(test.invoke("안녕하세요"))
print(test.invoke("VTW는 어떤 회사입니까?"))


datasource='own-generating'
datasource='vectorstore'
